In [2]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import itertools

In [3]:
## initiate file paths
screening_results_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/all_results_for_tableau.xlsx"
sample_points_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/Sampling_Sites_MASTER.xlsx"

In [4]:
sample_pts = pd.read_excel(sample_points_path, sheet_name = "Sample_Locations")
screening_results = pd.read_excel(screening_results_path, sheet_name = "Sheet1")

In [5]:
sample_pts_gdf = gp.GeoDataFrame(sample_pts, geometry=gp.points_from_xy(sample_pts.Longitude, sample_pts.Latitude), crs="EPSG:4326")

In [6]:
sample_pts_gdf.columns

Index(['Sampling ID', 'Site ID', 'DateTime', 'Date', 'Medium',
       'Number of Times Sampled', 'Latitude', 'Longitude', 'Description',
       'Photo', 'Issues Peter has Found', 'Resolved', 'geometry'],
      dtype='object')

In [7]:
screening_results['DATE']
screening_results["DATE"] = pd.to_datetime(screening_results["DATE"])

In [8]:
sample_pts_gdf['Date']
sample_pts_gdf["Date"] = pd.to_datetime(sample_pts_gdf["Date"], format='mixed')

In [12]:
sample_pts_join_results = pd.merge(screening_results, sample_pts_gdf, left_on = ['Sample ID', 'DATE'], right_on = ['Sampling ID', 'Date'], how = 'left')
sample_pts_join_results.rename(columns = {'Medium_x':'Medium'}, inplace = True)

In [21]:
columns = ['Medium', 'DATE', 'Sample ID', 'Site ID','Chemical Group',
       'Chemical', 'Result Parameter Name', 'Screening Level Type',
       'Screening Level Measurement', 'Result Value', 'Unit', 'Source',
       'SL_exceeded', 'Result Data Qualifier','Latitude',
       'Longitude','geometry']

sample_pts_join_results_gdf = gp.GeoDataFrame(sample_pts_join_results)
sample_pts_join_results_gdf['DATE'] = sample_pts_join_results_gdf['DATE'].astype(str)
sample_pts_join_results_gdf[columns].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/screening_results_join_geometry.shp")

/tmp/ipykernel_228/2715928309.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sample_pts_join_results_gdf[columns].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/screening_results_join_geometry.shp")
